# TFIDF + Logistic Regression

In [ ]:

# Imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing

# Combine train and dev for vectorizer fitting
full_train_texts = train_df['text'].tolist() + dev_df['text'].tolist()
full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

# Mapping labels to ids
labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',   # character n-grams inside word boundaries
    ngram_range=(2,6),    # 2-6 character n-grams
    max_features=50000    # limit vocabulary size
)

# Fit on training + dev set
tfidf.fit(full_train_texts)

# Transform
X_train = tfidf.transform(train_df['text'])
X_dev = tfidf.transform(dev_df['text'])
X_test = tfidf.transform(test_df['text'])

# Train Logistic Regression
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Evaluate on Dev Set
dev_preds = clf.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score: {dev_macro_f1:.4f}")

Validation Macro F1 Score: 0.4114
Saved Track 1 submission to outputs/track_1_test.csv


In [ ]:
# Predict on Test Set
test_preds = clf.predict(X_test)
test_labels = [id2label[pred] for pred in test_preds]

# Save to CSV
output_df = pd.DataFrame({
    'id': test_df['id'],
    'label': test_labels
})
os.makedirs('outputs', exist_ok=True)
output_df.to_csv('outputs/track_1_test.csv', index=False)

print("Saved Track 1 submission to outputs/track_1_test.csv")

# TFIDF + Elastic Net

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from tqdm import tqdm
import os
import pickle

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing
full_train_texts = train_df['text'].tolist() + dev_df['text'].tolist()
full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2,6),
    max_features=50000
)

tfidf.fit(full_train_texts)

X_train = tfidf.transform(train_df['text'])
X_dev = tfidf.transform(dev_df['text'])
X_test = tfidf.transform(test_df['text'])


# Too time consuming TODO: Try with cv before submitting
# # Hyperparameter Grid
# param_grid = {
#     'C': [0.01, 10],
#     'l1_ratio': [0.3, 0.7]
# }
# grid = list(ParameterGrid(param_grid))

# # Cache file
# CACHE_FILE = './cache/ENET_CV5_cache.pkl'

# # Load previous cache if exists
# if os.path.exists(CACHE_FILE):
#     with open(CACHE_FILE, 'rb') as f:
#         cache = pickle.load(f)
#     print(f"Loaded {len(cache)} cached results.")
# else:
#     cache = {}

# best_score = 0
# best_params = None

# # Define scoring
# scorer = make_scorer(f1_score, average='macro')

# # Search
# for params in tqdm(grid, desc="Grid Search with 5-Fold CV"):
#     param_key = tuple(sorted(params.items()))  # make a hashable key

#     if param_key in cache:
#         macro_f1 = cache[param_key]
#     else:
#         print(f'Start of parameter tuning: {param_key}')
#         model = LogisticRegression(
#             penalty='elasticnet',
#             solver='saga',
#             max_iter=1000,
#             random_state=42,
#             n_jobs=-1,
#             **params
#         )
#         # Perform 5-Fold CV here!
#         scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer, n_jobs=-1)
#         macro_f1 = np.mean(scores)

#         cache[param_key] = macro_f1

#         # Save cache every time
#         with open(CACHE_FILE, 'wb') as f:
#             pickle.dump(cache, f)
#         print(f'Cached {param_key}, f1: {macro_f1:.4f}')

#     if macro_f1 > best_score:
#         best_score = macro_f1
#         best_params = params

# print(f"Best parameters: {best_params}")
# print(f"Best Macro F1 Score (5-Fold CV): {best_score:.4f}")

# Train the best model fully
best_model = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    C=1,
    l1_ratio= 0.3
    # **best_params
)
print("Fitting model")
best_model.fit(X_train, y_train)

# Evaluate again on Dev Set
print("Evaluation model")
dev_preds = best_model.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score (retrained best model): {dev_macro_f1:.4f}")
# C=1, L1_ratio=0.7 -> 0.4172
# C=1, L1_ratio=0.5 -> 0.4222
# C=0.5, L1_ratio=0.5 -> 0.4024
# C=1, L1_ratio=0.3 -> 0.4261


Fitting model
Evaluation model
Validation Macro F1 Score (retrained best model): 0.4261


In [ ]:
# Predict on Test Set
test_preds = clf.predict(X_test)
test_labels = [id2label[pred] for pred in test_preds]

# Save to CSV
output_df = pd.DataFrame({
    'id': test_df['id'],
    'label': test_labels
})
os.makedirs('outputs', exist_ok=True)
output_df.to_csv('outputs/track_1_test.csv', index=False)

print("Saved Track 1 submission to outputs/track_1_test.csv")

# Word2Vec + XGBoost

In [ ]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300') 

In [8]:
import gensim.downloader as api
glove_twitter = api.load('glove-twitter-200')  

[==================================================] 100.0% 758.5/758.5MB downloaded


In [13]:
fast_wiki = api.load('fasttext-wiki-news-subwords-300')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb
from tqdm import tqdm
import os

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing

# Combine train and dev for label mapping
full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

# Map labels to integers
labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# Load pre-trained Word2Vec embeddings
print("Loading Word2Vec model...")
w2v = fast_wiki # This is 300-dim Google News vectors
print("Word2Vec model loaded.")

embedding_dim = 300

# Function to vectorize text
def text_to_vector(text, model, dim):
    words = text.split()
    word_vectors = []
    for word in words:
        if word in model:
            word_vectors.append(model[word])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(dim)

# Vectorize train, dev, and test sets
print("Vectorizing texts...")

X_train = np.vstack([text_to_vector(text, w2v, embedding_dim) for text in tqdm(train_df['text'])])
X_dev = np.vstack([text_to_vector(text, w2v, embedding_dim) for text in tqdm(dev_df['text'])])
X_test = np.vstack([text_to_vector(text, w2v, embedding_dim) for text in tqdm(test_df['text'])])

print("Vectorization complete.")

# Train XGBoost Classifier
print("Training XGBoost...")

model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(labels),
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# Evaluate on Dev Set
dev_preds = model.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score: {dev_macro_f1:.4f}")

# # Predict on Test Set
# test_preds = model.predict(X_test)
# test_labels = [id2label[pred] for pred in test_preds]

# # Save submission
# output_df = pd.DataFrame({
#     'id': test_df['id'],
#     'label': test_labels
# })
# os.makedirs('outputs', exist_ok=True)
# output_df.to_csv('outputs/track_3_test.csv', index=False)

# print("Saved Track 3 submission to outputs/track_3_test.csv")

"""
n_estimators=300,
learning_rate=0.1,
max_depth=6,
-> 0.3628

n_estimators=500,
learning_rate=0.5,
max_depth=8,
-> 0.3645

Twitter
n_estimators=300,
learning_rate=0.1,
max_depth=6,
-> 0.3235

Fast
n_estimators=300,
learning_rate=0.1,
max_depth=6,
-> 0.3570



'n_estimators': [300, 500],
'max_depth': [4, 6, 8],
'learning_rate': [0.01, 0.05, 0.1],
'subsample': [0.7, 0.8, 1.0],
'colsample_bytree': [0.7, 0.8, 1.0]
"""


Loading Word2Vec model...
Word2Vec model loaded.
Vectorizing texts...


100%|██████████| 4000/4000 [00:00<00:00, 10453.97it/s]


Vectorization complete.
Training XGBoost...
Validation Macro F1 Score: 0.3570


"\nn_estimators=300,\nlearning_rate=0.1,\nmax_depth=6,\n-> 0.3628\n\nTwitter\nn_estimators=300,\nlearning_rate=0.1,\nmax_depth=6,\n-> 0.3235\n\n\nn_estimators=500,\nlearning_rate=0.5,\nmax_depth=8,\n-> 0.3645\n\n'n_estimators': [300, 500],\n'max_depth': [4, 6, 8],\n'learning_rate': [0.01, 0.05, 0.1],\n'subsample': [0.7, 0.8, 1.0],\n'colsample_bytree': [0.7, 0.8, 1.0]\n"

In [ ]:
import pandas as pd
import numpy as np
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import xgboost as xgb
from tqdm import tqdm
import os
import pickle

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing

full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# Load pre-trained Word2Vec embeddings
print("Loading Word2Vec model...")
w2v = api.load('word2vec-google-news-300')  # Pretrained 300d embeddings
print("Word2Vec model loaded.")

embedding_dim = 300

# Function to vectorize text
def text_to_vector(text, model, dim):
    words = text.split()
    word_vectors = []
    for word in words:
        if word in model:
            word_vectors.append(model[word])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(dim)

# Vectorize train, dev, and test sets (with Cache)
CACHE_FOLDER = 'cache'
os.makedirs(CACHE_FOLDER, exist_ok=True)

def get_cached_vectors(filename, texts):
    filepath = os.path.join(CACHE_FOLDER, filename)
    if os.path.exists(filepath):
        print(f"Loading cached vectors from {filepath}...")
        return np.load(filepath)
    else:
        print(f"Vectorizing and caching {filename}...")
        vectors = np.vstack([text_to_vector(text, w2v, embedding_dim) for text in tqdm(texts)])
        np.save(filepath, vectors)
        return vectors

X_train = get_cached_vectors('train_vectors.npy', train_df['text'])
X_dev = get_cached_vectors('dev_vectors.npy', dev_df['text'])
X_test = get_cached_vectors('test_vectors.npy', test_df['text'])

print("Vectorization complete.")

# Cache or train best XGBoost model
BEST_MODEL_CACHE = os.path.join(CACHE_FOLDER, 'xgb_best_model.pkl')

if os.path.exists(BEST_MODEL_CACHE):
    print(f"Loading best model from cache {BEST_MODEL_CACHE}...")
    with open(BEST_MODEL_CACHE, 'rb') as f:
        best_model = pickle.load(f)
else:
    print("Starting hyperparameter tuning with GridSearchCV...")

    xgb_model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=len(labels),
        random_state=42,
        n_jobs=-1,
        verbosity=0
    )

    param_grid = {
        'n_estimators': [100, 300],
        '   ': [0.05, 0.1],
        'max_depth': [4, 6, 8]
    }

    grid = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        cv=5,
        scoring='f1_macro',
        verbose=2,
        n_jobs=-1
    )

    grid.fit(X_train, y_train)

    print(f"Best Params: {grid.best_params_}")
    print(f"Best CV Macro F1: {grid.best_score_:.4f}")

    best_model = grid.best_estimator_

    # Save best model to cache
    with open(BEST_MODEL_CACHE, 'wb') as f:
        pickle.dump(best_model, f)
    print(f"Saved best model to {BEST_MODEL_CACHE}")

# Evaluate on Dev Set
dev_preds = best_model.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score (best model): {dev_macro_f1:.4f}")

# Predict on Test Set
test_preds = best_model.predict(X_test)
test_labels = [id2label[pred] for pred in test_preds]

# Save submission
output_df = pd.DataFrame({
    'id': test_df['id'],
    'label': test_labels
})
os.makedirs('outputs', exist_ok=True)
output_df.to_csv('outputs/track_3_test.csv', index=False)

print("Saved Track 3 submission to outputs/track_3_test.csv")


# TFIDF + XGBoost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb
from tqdm import tqdm
import os

# Load data
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')
test_df = pd.read_csv('../data/test.csv')

# Preprocessing
full_train_texts = train_df['text'].tolist() + dev_df['text'].tolist()
full_train_labels = train_df['label'].tolist() + dev_df['label'].tolist()

labels = list(sorted(set(full_train_labels)))
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for label, idx in label2id.items()}

y_train = [label2id[label] for label in train_df['label']]
y_dev = [label2id[label] for label in dev_df['label']]

# TF-IDF Vectorization
tfidf = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2,6),
    max_features=50000
)

print("Vectorizing data")
tfidf.fit(full_train_texts)

X_train = tfidf.transform(train_df['text'])
X_dev = tfidf.transform(dev_df['text'])
X_test = tfidf.transform(test_df['text'])
print("Vectorization complete.")

# Train XGBoost Classifier
print("Training XGBoost...")

model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(labels),
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbosity=1
)

eval_set = [(X_train, y_train), (X_dev, y_dev)]

model.fit(
    X_train, y_train,
    eval_set=eval_set,
    # eval_metric='mlogloss',
    # early_stopping_rounds=10,  # ⛔ Stop if no dev improvement in 10 rounds
    verbose=True
)

# Dev Set Evaluation
print("Evaluating on dev set...")
dev_preds = model.predict(X_dev)
dev_macro_f1 = f1_score(y_dev, dev_preds, average='macro')
print(f"Validation Macro F1 Score: {dev_macro_f1:.4f}")

# # Predict on Test Set
# test_preds = model.predict(X_test)
# test_labels = [id2label[pred] for pred in test_preds]

# # Save submission
# output_df = pd.DataFrame({
#     'id': test_df['id'],
#     'label': test_labels
# })
# os.makedirs('outputs', exist_ok=True)
# output_df.to_csv('outputs/track_3_test.csv', index=False)

# print("Saved Track 3 submission to outputs/track_3_test.csv")

"""
'n_estimators': [300, 500],
'max_depth': [4, 6, 8],
'learning_rate': [0.01, 0.05, 0.1],
'subsample': [0.7, 0.8, 1.0],
'colsample_bytree': [0.7, 0.8, 1.0]
"""

# score = 0.3934

Vectorizing data
Vectorization complete.
Training XGBoost...
[0]	validation_0-mlogloss:1.59660	validation_1-mlogloss:1.59694
[1]	validation_0-mlogloss:1.58542	validation_1-mlogloss:1.58588
[2]	validation_0-mlogloss:1.57677	validation_1-mlogloss:1.57749
[3]	validation_0-mlogloss:1.56816	validation_1-mlogloss:1.56938
[4]	validation_0-mlogloss:1.56004	validation_1-mlogloss:1.56188
[5]	validation_0-mlogloss:1.55268	validation_1-mlogloss:1.55508
[6]	validation_0-mlogloss:1.54618	validation_1-mlogloss:1.54884
[7]	validation_0-mlogloss:1.54010	validation_1-mlogloss:1.54338
[8]	validation_0-mlogloss:1.53430	validation_1-mlogloss:1.53803
[9]	validation_0-mlogloss:1.52921	validation_1-mlogloss:1.53344
[10]	validation_0-mlogloss:1.52471	validation_1-mlogloss:1.52969
[11]	validation_0-mlogloss:1.52035	validation_1-mlogloss:1.52629
[12]	validation_0-mlogloss:1.51612	validation_1-mlogloss:1.52260
[13]	validation_0-mlogloss:1.51210	validation_1-mlogloss:1.51908
[14]	validation_0-mlogloss:1.50845	vali

"\n'n_estimators': [300, 500],\n'max_depth': [4, 6, 8],\n'learning_rate': [0.01, 0.05, 0.1],\n'subsample': [0.7, 0.8, 1.0],\n'colsample_bytree': [0.7, 0.8, 1.0]\n"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import xgboost as xgb
import joblib
import pickle
import os

SAVE_DIR = "saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# ========== Utility: Check if all models exist ==========
def models_exist():
    required_files = [
        "tfidf_vectorizer.pkl",
        "logreg_model.pkl",
        "xgb_model.pkl",
        "meta_model.pkl",
        "label_maps.pkl"
    ]
    return all(os.path.exists(os.path.join(SAVE_DIR, fname)) for fname in required_files)

# ========== Load Data ==========
train_df = pd.read_csv('../data/train.csv')
dev_df = pd.read_csv('../data/dev.csv')

# ========== Train + Save Models If Not Exist ==========
if not models_exist():
    print("Models not found. Training models...")

    # Label Encoding
    all_labels = sorted(set(train_df['label']) | set(dev_df['label']))
    label2id = {label: i for i, label in enumerate(all_labels)}
    id2label = {i: label for label, i in label2id.items()}
    y_train = train_df['label'].map(label2id).values
    y_dev = dev_df['label'].map(label2id).values

    # TF-IDF Vectorization
    print("Fitting TF-IDF vectorizer...")
    tfidf = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 6), max_features=50000)
    tfidf.fit(pd.concat([train_df['text'], dev_df['text']]))

    X_train = tfidf.transform(train_df['text'])
    X_dev = tfidf.transform(dev_df['text'])

    # Train base models
    print("Training Logistic Regression...")
    logreg = LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        max_iter=1000,
        C=1.0,
        l1_ratio=0.3,
        random_state=42,
        n_jobs=-1
    )
    logreg.fit(X_train, y_train)

    print("Training XGBoost...")
    xgb_clf = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(label2id),
        n_estimators=300,
        learning_rate=0.1,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    xgb_clf.fit(X_train, y_train)

    # Generate meta-features (dev set)
    print("Generating meta-features...")
    dev_preds_logreg = logreg.predict_proba(X_dev)
    dev_preds_xgb = xgb_clf.predict_proba(X_dev)
    X_meta_dev = np.hstack([dev_preds_logreg, dev_preds_xgb])

    # Train meta-model
    print("Training meta-model...")
    meta_model = LogisticRegression(max_iter=1000, random_state=42)
    meta_model.fit(X_meta_dev, y_dev)

    # Evaluate
    meta_dev_preds = meta_model.predict(X_meta_dev)
    meta_f1 = f1_score(y_dev, meta_dev_preds, average='macro')
    print(f"Stacked Dev Macro F1: {meta_f1:.4f}")

    # Save models
    print("Saving models...")
    with open(os.path.join(SAVE_DIR, "tfidf_vectorizer.pkl"), "wb") as f:
        pickle.dump(tfidf, f)
    joblib.dump(logreg, os.path.join(SAVE_DIR, "logreg_model.pkl"))
    joblib.dump(xgb_clf, os.path.join(SAVE_DIR, "xgb_model.pkl"))
    joblib.dump(meta_model, os.path.join(SAVE_DIR, "meta_model.pkl"))
    with open(os.path.join(SAVE_DIR, "label_maps.pkl"), "wb") as f:
        pickle.dump({'label2id': label2id, 'id2label': id2label}, f)

    print("Models saved.") # used 38 mins
else:
    print("Saved models already exist. Skipping training.")


Models not found. Training models...
Fitting TF-IDF vectorizer...
Training Logistic Regression...
Training XGBoost...
Generating meta-features...
Training meta-model...
Stacked Dev Macro F1: 0.4316
Saving models...
Models saved.
